In [1]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv(r'C:\Users\asus\Cursor Code\Day1_NLP_TweetDisaster\Dataset\Data_day1_DisasterTweets\train.csv')

In [3]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

## Pembuatan panjang kata untuk setiap fitur


In [5]:
df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" ")))
df[['text','word_count']].head()

,text,word_count
0,Our Deeds are the Reason of this #earthquake M...,13
1,Forest fire near La Ronge Sask. Canada,7
2,All residents asked to 'shelter in place' are ...,22
3,"13,000 people receive #wildfires evacuation or...",9
4,Just got sent this photo from Ruby #Alaska as ...,17


In [6]:
df['length']=df['text'].apply(lambda x: len(str(x)))

In [7]:
df[['text','word_count','length']]

,text,word_count,length
0,Our Deeds are the Reason of this #earthquake M...,13,69
1,Forest fire near La Ronge Sask. Canada,7,38
2,All residents asked to 'shelter in place' are ...,22,133
3,"13,000 people receive #wildfires evacuation or...",9,65
4,Just got sent this photo from Ruby #Alaska as ...,17,88
...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,11,83
7609,@aria_ahrary @TheTawniest The out of control w...,20,125
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,8,65
7611,Police investigating after an e-bike collided ...,19,137


In [8]:
df.describe()

,id,target,word_count,length
count,7613.000000,7613.00000,7613.000000,7613.000000
mean,5441.934848,0.42966,14.928937,101.037436
std,3137.116090,0.49506,5.782770,33.781325
min,1.000000,0.00000,1.000000,7.000000
25%,2734.000000,0.00000,11.000000,78.000000
50%,5408.000000,0.00000,15.000000,107.000000
75%,8146.000000,1.00000,19.000000,133.000000
max,10873.000000,1.00000,54.000000,157.000000


In [9]:
df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

## Preprocess

In [10]:
df=df.fillna("")

In [11]:
df.head()

,id,keyword,location,text,target,word_count,length
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,13,69
1,4,,,Forest fire near La Ronge Sask. Canada,1,7,38
2,5,,,All residents asked to 'shelter in place' are ...,1,22,133
3,6,,,"13,000 people receive #wildfires evacuation or...",1,9,65
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,17,88


## Pembagian data

In [12]:
X=df['text']
y=df['target']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6090,), (1523,), (6090,), (1523,))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(stop_words='english',max_features=5000,ngram_range=(1,2))

## Coba pake machine learning tradisional


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [17]:
X_train_tfidf=vectorizer.fit_transform(X_train)
X_test_tfidf=vectorizer.transform(X_test)
models = {
    "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42),
    "Multinomial Naive Bayes": MultinomialNB(),
    "Linear SVM": LinearSVC(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Loop through each model to train and evaluate it
for name, model in models.items():
    print(f"\n--- Training {name} ---")

    # Train the model on the TF-IDF transformed training data
    model.fit(X_train_tfidf, y_train)
    print(f"Training complete.")

    # --- 5. Evaluate the Model ---
    print(f"Evaluating {name}...")

    # Make predictions on the transformed test data
    predictions = model.predict(X_test_tfidf)

    # Print the performance report
    print(f"\nClassification Report for {name}:")
    print(classification_report(y_test, predictions, zero_division=0))
    print("-" * 50)


--- Training Logistic Regression ---
Training complete.
Evaluating Logistic Regression...

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       869
           1       0.84      0.71      0.77       654

    accuracy                           0.82      1523
   macro avg       0.82      0.81      0.81      1523
weighted avg       0.82      0.82      0.82      1523

--------------------------------------------------

--- Training Multinomial Naive Bayes ---
Training complete.
Evaluating Multinomial Naive Bayes...

Classification Report for Multinomial Naive Bayes:
              precision    recall  f1-score   support

           0       0.79      0.91      0.85       869
           1       0.86      0.68      0.76       654

    accuracy                           0.81      1523
   macro avg       0.82      0.80      0.80      1523
weighted avg       0.82      0.81      0.81      1523

-----

### Base model paling bagus di logisticregression pake ditune

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
param_grid = {
    'C': [0.1, 1, 5, 10],  # Nilai kekuatan regularisasi
    'solver': ['liblinear', 'lbfgs']
}
grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=42),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

In [20]:
grid_search.fit(X_train_tfidf, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


,estimator,LogisticRegre...ndom_state=42)
,param_grid,"{'C': [0.1, 1, ...], 'solver': ['liblinear', 'lbfgs']}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [21]:
print(f"Parameter terbaik yang ditemukan: {grid_search.best_params_}")
print(f"Skor cross-validation terbaik: {grid_search.best_score_:.4f}\n")

Parameter terbaik yang ditemukan: {'C': 1, 'solver': 'liblinear'}
Skor cross-validation terbaik: 0.7959



In [22]:
best_model = grid_search.best_estimator_

predictions = best_model.predict(X_test_tfidf)

print("Laporan Klasifikasi dari Model yang Sudah Dituning:")
print(classification_report(y_test, predictions))

Laporan Klasifikasi dari Model yang Sudah Dituning:
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       869
           1       0.84      0.71      0.77       654

    accuracy                           0.82      1523
   macro avg       0.82      0.81      0.81      1523
weighted avg       0.82      0.82      0.82      1523



### Nice naik ke 82% coba pake model deeplearning (pytorch)

In [28]:
import torch

# Set device globally: CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [36]:
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
vectorizer.fit(X)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,'english'
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [37]:
vocab = {word: i + 1 for word, i in vectorizer.vocabulary_.items()}
vocab['<unk>'] = 0 

## bikin pipeline buat teks

In [38]:
def text_pipeline(x):
    tokens = vectorizer.build_analyzer()(x)
    return [vocab.get(word, vocab['<unk>']) for word in tokens]

# Sekarang bikin Class dataset,dll

In [40]:
class DisasterTweetsDataset(Dataset):
    def __init__(self, text, label):
        self.text = text
        self.label = label
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        text=self.text[idx]
        label=self.label[idx]
        return torch.tensor(text_pipeline(text), dtype=torch.int64), torch.tensor(label, dtype=torch.int64)

In [41]:
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(_text, dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(len(processed_text))
    return torch.nn.utils.rnn.pad_sequence(text_list, padding_value=0, batch_first=True), \
           torch.tensor(label_list, dtype=torch.int64)

In [42]:
class TextClassifierLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super(TextClassifierLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, (hidden, _) = self.lstm(embedded)
        return self.fc(hidden.squeeze(0))

In [49]:
VOCAB_SIZE = len(vocab) # Ukuran vocab sekarang dari CountVectorizer
EMBED_DIM = 64
HIDDEN_DIM = 32
NUM_CLASS = 2
EPOCHS = 5
LR = 0.001
BATCH_SIZE = 32

# Instantiate model, loss, and optimizer
model = TextClassifierLSTM(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, NUM_CLASS)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)
train_dataset = DisasterTweetsDataset(X_train.tolist(), y_train.tolist())
test_dataset = DisasterTweetsDataset(X_test.tolist(), y_test.tolist())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [45]:
def train(model, dataloader):
    model.train()
    total_acc, total_count = 0, 0
    for texts, labels in dataloader:
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
    return total_acc / total_count

In [46]:
def evaluate(model, dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for texts, labels in dataloader:
            predictions = model(texts)
            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
    return total_acc / total_count

In [50]:
print("\nTraining model PyTorch LSTM dengan vocabulary dari CountVectorizer...")
for epoch in range(1, EPOCHS + 1):
    accu_train = train(model, train_loader)
    print(f'Epoch {epoch:02d}, Akurasi Training: {accu_train:.4f}')

accu_test = evaluate(model, test_loader)
print(f'\nAkurasi Test: {accu_test:.4f}')


Training model PyTorch LSTM dengan vocabulary dari CountVectorizer...


C:\Users\asus\AppData\Local\Temp\ipykernel_2648\2069240813.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(_text, dtype=torch.int64)


Epoch 01, Akurasi Training: 0.6228
Epoch 02, Akurasi Training: 0.7524
Epoch 03, Akurasi Training: 0.7993
Epoch 04, Akurasi Training: 0.8279
Epoch 05, Akurasi Training: 0.8404

Akurasi Test: 0.7919


### Sepertinya ada overfit pada kode pytorch

### perbedaan akurasi dengan LSTM dan logistic tidak beda jauh